In [2]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.6f}".format

train_df = pd.read_csv(filepath_or_buffer="raw_bitstrings/bitstrings_no_noise_100000.csv")
test_df = pd.read_csv(filepath_or_buffer="raw_bitstrings/bitstrings_no_noise_1000.csv")
validation_df = pd.read_csv(filepath_or_buffer="raw_bitstrings/bitstrings_no_noise_10000.csv")

train_df.head()

,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,...,9_95,9_96,9_97,9_98,9_99,decay_0,decay_1,W_0,W_1,J_0
0,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2.123473,2.101471,4.160667,2.129584,4.071764
1,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,2,0.557067,1.261318,2.288560,3.381280,2.231517
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,3,3,1.357797,-0.558892,3.235920,2.811693,3.467802
3,0,0,0,0,0,0,0,0,0,0,...,3,0,3,3,3,0.674877,-0.554495,2.702982,2.245201,2.668579
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1.253716,0.462385,2.295924,3.280174,4.145718


In [4]:
%load_ext tensorboard
import datetime
import tensorflow as tf
from custom_layers import CustomConnectedLayer, CustomConnectedLayer2

learning_rate = 0.001
epochs = 30
batch_size = 10000

# Get all column names as a list
keys = train_df.keys().tolist()

# Remove specific columns from the list
keys = [key for key in keys if key not in ['W_0', 'W_1', 'J_0']]

inputs = {key:tf.keras.layers.Input(shape=(1,), name=key) for key in keys}
concatenated_inputs = tf.keras.layers.concatenate(inputs.values())
# Prepare data for training
train_features = {key: train_df[key] for key in inputs}
train_labels = train_df[['W_0', 'W_1', 'J_0']]

# Similarly prepare test and validation data
test_features = {key: test_df[key] for key in inputs}
test_labels = test_df[['W_0', 'W_1', 'J_0']]
#
validation_features = {key: validation_df[key] for key in inputs}
validation_labels = validation_df[['W_0', 'W_1', 'J_0']]

input_shape = concatenated_inputs.shape

# Replace Dense layers with CustomConnectedLayer
hidden_layer_1 = CustomConnectedLayer(1000, 100, activation='relu')(concatenated_inputs)
hidden_layer_2 = tf.keras.layers.Dense(100, activation='relu')(hidden_layer_1)
hidden_layer_3 = tf.keras.layers.Dense(100, activation='relu')(hidden_layer_2)
hidden_layer_4 = tf.keras.layers.Dense(100, activation='relu')(hidden_layer_3)

#hidden_layer_2 = CustomConnectedLayer(100, num_groups=1, activation='relu')(hidden_layer_1)
#hidden_layer_3 = CustomConnectedLayer(64, num_groups=1, activation='relu')(hidden_layer_2)
#hidden_layer_4 = CustomConnectedLayer(64, num_groups=1, activation='relu')(hidden_layer_3)


output = tf.keras.layers.Dense(3)(hidden_layer_3) #TODO magic number

log_dir = "logs/gradient_tape/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model = tf.keras.Model(inputs=inputs, outputs=output)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mean_squared_error')


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:

# Train the model
history = model.fit(train_features, train_labels, validation_data=(validation_features, validation_labels), epochs=epochs, batch_size=batch_size, callbacks=[tensorboard_callback])
#history = model.fit(train_features, train_labels, validation_split=0.2, epochs=epochs, batch_size=batch_size)

# Evaluate the model
model.evaluate(test_features, test_labels)

# Extract loss and validation loss
train_loss = history.history['loss']
val_loss = history.history['val_loss']

Epoch 1/30

10/10 [==============================] - 12s 983ms/step - loss: 6.5284 - val_loss: 1.0859
Epoch 2/30
10/10 [==============================] - 6s 586ms/step - loss: 1.0454 - val_loss: 0.9813
Epoch 3/30
10/10 [==============================] - 5s 523ms/step - loss: 0.8522 - val_loss: 0.7621
Epoch 4/30
10/10 [==============================] - 5s 562ms/step - loss: 0.7224 - val_loss: 0.6899
Epoch 5/30
10/10 [==============================] - 6s 569ms/step - loss: 0.6414 - val_loss: 0.6226
Epoch 6/30
10/10 [==============================] - 5s 520ms/step - loss: 0.5890 - val_loss: 0.5824
Epoch 7/30
10/10 [==============================] - 5s 538ms/step - loss: 0.5582 - val_loss: 0.5618
Epoch 8/30
10/10 [==============================] - 6s 642ms/step - loss: 0.5410 - val_loss: 0.5490
Epoch 9/30
10/10 [==============================] - 5s 504ms/step - loss: 0.5300 - val_loss: 0.5410
Epoch 10/30
10/10 [==============================] - 5s 522ms/step - loss: 0.5226 - val_loss: 0.53

In [ ]:
epochs_range = range(epochs)

# Plotting
plt.figure(figsize=(8, 8))
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
#plt.xlim([1,epochs])
#plt.ylim([0,2])
plt.semilogy()
plt.show()

In [11]:
%tensorboard

Launching TensorBoard...

In [19]:
%tensorboard --logdir logs/gradient_tape

Launching TensorBoard...